#### Dynamic Group Chat - Selecting Agents Dynamically using LLM

In [1]:
from autogen_agentchat.agents import AssistantAgent, UserProxyAgent
from autogen_agentchat.conditions import TextMentionTermination, MaxMessageTermination
from autogen_agentchat.teams import SelectorGroupChat
from autogen_agentchat.ui import Console
from autogen_agentchat.messages import AgentEvent, ChatMessage
from autogen_ext.models.openai import AzureOpenAIChatCompletionClient
from autogen_agentchat.base import TaskResult

import os

from dotenv import load_dotenv

In [ ]:
load_dotenv(override=True)

azure_openai_api_key = os.getenv("AZURE_OPENAI_API_KEY")
azure_openai_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
azure_openai_deployment_name = os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME")
azure_openai_api_version = os.getenv("AZURE_OPENAI_API_VERSION")

if not all([azure_openai_api_key, azure_openai_endpoint, azure_openai_deployment_name, azure_openai_api_version]):
    raise ValueError("""
        Please set all required environment variables: 
            AZURE_OPENAI_API_KEY, 
            AZURE_OPENAI_ENDPOINT, 
            AZURE_OPENAI_DEPLOYMENT_NAME, 
            AZURE_OPENAI_API_VERSION
        """)


In [3]:
model_client = AzureOpenAIChatCompletionClient(
    azure_deployment=azure_openai_deployment_name,
    model=azure_openai_deployment_name,
    api_key=azure_openai_api_key,
    azure_endpoint=azure_openai_endpoint,
    api_version=azure_openai_api_version,
)

In [4]:
def weather_check_tool(location: str) -> str:
    weather_data = {
        "New York": "Sunny, 25°C",
        "Los Angeles": "Cloudy, 22°C",
        "Chicago": "Rainy, 18°C",
        "London": "Windy, 15°C",
        "Tokyo": "Clear, 28°C",
        "Sydney": "Sunny, 30°C",
        "Berlin": "Overcast, 20°C",
        "Paris": "Partly cloudy, 23°C",
        "Moscow": "Snowy, -5°C",
    }

    return weather_data.get(location, "Weather data not available for this location.")


def currency_exchange_tool(amount: float, from_currency: str, to_currency: str) -> str:
    exchange_rates = {
        "USD": {"EUR": 0.85, "JPY": 110.0, "GBP": 0.75},
        "EUR": {"USD": 1.18, "JPY": 130.0, "GBP": 0.88},
        "JPY": {"USD": 0.0091, "EUR": 0.0077, "GBP": 0.0066},
        "GBP": {"USD": 1.33, "EUR": 1.14, "JPY": 150.0},
        "AUD": {"USD": 0.74, "EUR": 0.63, "JPY": 82.0},
        "CAD": {"USD": 0.79, "EUR": 0.67, "JPY": 88.0},
        "CHF": {"USD": 1.09, "EUR": 0.93, "JPY": 120.0},
        "CNY": {"USD": 0.15, "EUR": 0.13, "JPY": 18.0},
    }

    if from_currency in exchange_rates and to_currency in exchange_rates[from_currency]:
        converted_amount = amount * exchange_rates[from_currency][to_currency]
        return f"{amount} {from_currency} is approximately {converted_amount:.2f} {to_currency}."
    else:
        return "Currency conversion not available for the specified currencies."

In [5]:
planning_system_prompt = """
        You are a planning agent.
        Your Job is to break down complex tasks into smaller and managable subtasks.
        Your Team Members are:
        - WeatherAgent: Provides weather information for specified locations.
        - CurrencyAgent: Provides currency exchange rates and conversions.
        
        You shall only plan and delegate tasks to your team members.
        You will not execute any tasks yourself.
        
        After all tasks are complete, summarize the results in a single message, and end with 'TERMINATE'.
        
        When assigning tasks, use this format.
        1. <agent_name>: <task_description>
        
        For example:
        1. WeatherAgent: Get the weather for New York.
"""

planning_agent = AssistantAgent(
    name="PlanningAgent",
    model_client=model_client,
    system_message=planning_system_prompt,
    description="An agent that plans tasks based on user requests and available tools.",
)

weather_agent = AssistantAgent(
    name="WeatherAgent",
    model_client=model_client,
    system_message="You are a weather information agent. Provide weather details for specified locations.",
    description="An agent that provides weather information for specified locations.",
    tools=[weather_check_tool],
)

currency_agent = AssistantAgent(
    name="CurrencyAgent",
    model_client=model_client,
    system_message="You are a currency exchange agent. Provide currency conversion rates and calculations.",
    description="An agent that provides currency exchange rates and conversions.",
    tools=[currency_exchange_tool],
)

In [7]:
from typing import Sequence


def custom_selector_func(messages: Sequence[AgentEvent | ChatMessage]) -> str:
    if messages[-1].source != planning_agent.name:
        return planning_agent.name

    return None

In [8]:
text_mention_termination = TextMentionTermination("TERMINATE")
max_message_termination = MaxMessageTermination(10)
combined_termination = text_mention_termination | max_message_termination

customer_service_team = SelectorGroupChat(
    [planning_agent, weather_agent, currency_agent],
    selector_func = custom_selector_func,
    termination_condition=combined_termination,
    model_client=model_client,
)

In [9]:
task = "How much is 100 USD in EUR? and What's the weather in New York and "

await Console(
    customer_service_team.run_stream(task=task))

---------- TextMessage (user) ----------
How much is 100 USD in EUR? and What's the weather in New York and 
---------- TextMessage (PlanningAgent) ----------
1. CurrencyAgent: Convert 100 USD to EUR.
2. WeatherAgent: Get the weather for New York.
---------- ToolCallRequestEvent (CurrencyAgent) ----------
[FunctionCall(id='call_SfzfbQzGIeAuW4PIl6W5RNAt', arguments='{"amount": 100, "from_currency": "USD", "to_currency": "EUR"}', name='currency_exchange_tool'), FunctionCall(id='call_ZtWz7weP55Rmega1ADWvA2t5', arguments='{"amount": 100, "from_currency": "USD", "to_currency": "EUR"}', name='currency_exchange_tool')]
---------- ToolCallExecutionEvent (CurrencyAgent) ----------
[FunctionExecutionResult(content='100.0 USD is approximately 85.00 EUR.', name='currency_exchange_tool', call_id='call_SfzfbQzGIeAuW4PIl6W5RNAt', is_error=False), FunctionExecutionResult(content='100.0 USD is approximately 85.00 EUR.', name='currency_exchange_tool', call_id='call_ZtWz7weP55Rmega1ADWvA2t5', is_error=Fa

TaskResult(messages=[TextMessage(source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 6, 12, 21, 32, 10, 13537, tzinfo=datetime.timezone.utc), content="How much is 100 USD in EUR? and What's the weather in New York and ", type='TextMessage'), TextMessage(source='PlanningAgent', models_usage=RequestUsage(prompt_tokens=170, completion_tokens=25), metadata={}, created_at=datetime.datetime(2025, 6, 12, 21, 32, 12, 191542, tzinfo=datetime.timezone.utc), content='1. CurrencyAgent: Convert 100 USD to EUR.\n2. WeatherAgent: Get the weather for New York.', type='TextMessage'), ToolCallRequestEvent(source='CurrencyAgent', models_usage=RequestUsage(prompt_tokens=132, completion_tokens=67), metadata={}, created_at=datetime.datetime(2025, 6, 12, 21, 32, 14, 243871, tzinfo=datetime.timezone.utc), content=[FunctionCall(id='call_SfzfbQzGIeAuW4PIl6W5RNAt', arguments='{"amount": 100, "from_currency": "USD", "to_currency": "EUR"}', name='currency_exchange_tool'), FunctionCall